In [5]:
from trellocore import TrelloCore
import os
import json

trellocore = None
with open(os.path.join(os.path.abspath(''),"ENV",".env"),"r") as file:
    key =json.load(file)
    trellocore = TrelloCore(app_key=key["api_key"], token=key["admin_token"])


In [6]:
member_id = trellocore.token().get_member()['id']
boards = trellocore.member().get_boards(id=member_id, fields="id,name")
print(boards)
ufficio = [board['id'] for board in boards if "UFFICIO" == board['name']][0]
print(ufficio)

[{'id': '5ee8c73b5fc8a0081dea8d11', 'name': 'Andrea'}, {'id': '6009703d968abd13ebdfbc27', 'name': 'Antje'}, {'id': '6007eb661419e5662e28d18d', 'name': 'Clarice'}, {'id': '5ee8c71a3aea4f250bffe6b2', 'name': 'Claudio'}, {'id': '5fdf45fd573e6a4950396ea1', 'name': 'FALEGNAMERIA'}, {'id': '5ee86a31b4dd441463ba65d6', 'name': 'Luca'}, {'id': '5f116945ac22ae6cd2d07294', 'name': 'Luca Valentini'}, {'id': '5ee8c7329f86d41a917505f5', 'name': 'Manuel'}, {'id': '5ee869db1e066a18aec6ba98', 'name': 'Marco'}, {'id': '5f11913437db9819e7f923c8', 'name': 'Nicola Miori'}, {'id': '5fdf45f03920b84a4c0eb520', 'name': 'UFFICIO'}, {'id': '5ee8c726d3314a3af9a9290f', 'name': 'Valentina'}, {'id': '5f02e306e2d1536d1c293e42', 'name': 'Vincenzo'}, {'id': '5f119143119bf0893fbf74d0', 'name': 'Virgil'}]
5fdf45f03920b84a4c0eb520


In [7]:
board = trellocore.board(id=ufficio)
#print(board_id)
liste = board.get_lists(fields="id,name", filter='open')
#[print(elemento) for elemento in liste]
bacheche_personali = [name['id'] for name in liste if "bacheche personali" == name['name'] ]
print(bacheche_personali)

['6008770254c19e8489d9f896']


In [8]:
cards = trellocore.list(id=bacheche_personali[0]).get_cards()
cards = [elemento['name'].split(':') for elemento in cards][1:]
bacheche_personali = [{'username': elemento[0],'board_name': elemento[1] } for elemento in cards]
print(bacheche_personali)

[{'username': 'andreagarbellotti', 'board_name': 'Andrea'}, {'username': 'antje416', 'board_name': 'Antje'}, {'username': 'claricecloutier', 'board_name': 'Clarice'}, {'username': 'zaninelli', 'board_name': 'Luca'}, {'username': 'manueltolotti', 'board_name': 'Manuel'}, {'username': 'marcozaninelli', 'board_name': 'Marco'}, {'username': 'valentinatasin', 'board_name': 'Valentina'}, {'username': 'vincenzolunelli', 'board_name': 'Vincenzo'}]


In [9]:

for elemento in bacheche_personali: 
    elemento['id'] = (trellocore.member(id=elemento['username']).get_member(fields="id")['id'])




In [10]:
for elemento in bacheche_personali:
    if elemento['username'] == 'zaninelli':
      elemento['type'] = 1
      elemento['token'] = key["admin_token"]
    else:
        elemento['type'] = 0
        elemento['token'] = None
    print(elemento)

{'username': 'andreagarbellotti', 'board_name': 'Andrea', 'id': '5ed9de3a44fb3e880f78e84d', 'type': 0, 'token': None}
{'username': 'antje416', 'board_name': 'Antje', 'id': '5ed80cbe514b035cfc99838a', 'type': 0, 'token': None}
{'username': 'claricecloutier', 'board_name': 'Clarice', 'id': '5c004b9ebb81ab0403618dc1', 'type': 0, 'token': None}
{'username': 'zaninelli', 'board_name': 'Luca', 'id': '5eda6809df259163876f4896', 'type': 1, 'token': '79f8f30cb4eb3e4d1fb4a689316365dd17a0fac5142d4ba7434389ce33cc13fb'}
{'username': 'manueltolotti', 'board_name': 'Manuel', 'id': '5edbcc7b78d55673cd5bda17', 'type': 0, 'token': None}
{'username': 'marcozaninelli', 'board_name': 'Marco', 'id': '5ed8067e73403d50623f87fd', 'type': 0, 'token': None}
{'username': 'valentinatasin', 'board_name': 'Valentina', 'id': '5ed8f7cfaaaef0624bcb4200', 'type': 0, 'token': None}
{'username': 'vincenzolunelli', 'board_name': 'Vincenzo', 'id': '5efed0b3bdfd1b6b709ff513', 'type': 0, 'token': None}


In [11]:
for elemento in bacheche_personali:
    for board in boards:
        if board['name'] == elemento['board_name']:
            elemento['board_id'] =  board['id']
            #print(elemento)


In [12]:
for elemento in bacheche_personali:
    board = trellocore.board(id=elemento['board_id'])
    liste = board.get_lists(fields="id,name", filter='open')

    elemento['list_id'] = [name['id'] for name in liste if "MANAGMEENT" == name['name'] ]

print(bacheche_personali[0].keys())

dict_keys(['username', 'board_name', 'id', 'type', 'token', 'board_id', 'list_id'])


In [13]:
import sqlite3
import os
db_file = os.path.join(os.path.abspath(''),"ENV", "database.sqlite3")
conn = sqlite3.connect(db_file)
cur = conn.cursor()

In [14]:
query = """DROP TABLE Employee;"""
cur.execute(query)

In [15]:
query = """
		CREATE TABLE IF NOT EXISTS Employee (
			id CHAR(24),
			username CHAR(50),
			board_id CHAR(24),
            board_name CHAR(50),
			list_id CHAR(24),
			token CHAR(50),
            type INT,
			PRIMARY KEY (id)
		);
        """
cur.execute(query)

In [16]:
for line in bacheche_personali:
    print(line)

{'username': 'andreagarbellotti', 'board_name': 'Andrea', 'id': '5ed9de3a44fb3e880f78e84d', 'type': 0, 'token': None, 'board_id': '5ee8c73b5fc8a0081dea8d11', 'list_id': ['602bb2415db587685a99298b']}
{'username': 'antje416', 'board_name': 'Antje', 'id': '5ed80cbe514b035cfc99838a', 'type': 0, 'token': None, 'board_id': '6009703d968abd13ebdfbc27', 'list_id': ['602bb24175564f7418415c26']}
{'username': 'claricecloutier', 'board_name': 'Clarice', 'id': '5c004b9ebb81ab0403618dc1', 'type': 0, 'token': None, 'board_id': '6007eb661419e5662e28d18d', 'list_id': ['602bb2424c48d5706dda9799']}
{'username': 'zaninelli', 'board_name': 'Luca', 'id': '5eda6809df259163876f4896', 'type': 1, 'token': '79f8f30cb4eb3e4d1fb4a689316365dd17a0fac5142d4ba7434389ce33cc13fb', 'board_id': '5ee86a31b4dd441463ba65d6', 'list_id': ['602bb244575c6775f9a91283']}
{'username': 'manueltolotti', 'board_name': 'Manuel', 'id': '5edbcc7b78d55673cd5bda17', 'type': 0, 'token': None, 'board_id': '5ee8c7329f86d41a917505f5', 'list_id'

In [17]:
for line in bacheche_personali:
    sql = f"""
    INSERT INTO Employee(id, username, board_id, board_name, list_id, token, type)
    VALUES(?,?,?,?,?,?,?);
    """
    value =(line['id'],line['username'],line['board_id'],line['board_name'],line['list_id'][0],line['token'],line['type'])
    cur.execute(sql,value)

In [18]:
select = "SELECT * FROM Employee;"

cur.execute(select)
print(cur.fetchall())

[('5ed9de3a44fb3e880f78e84d', 'andreagarbellotti', '5ee8c73b5fc8a0081dea8d11', 'Andrea', '602bb2415db587685a99298b', None, 0), ('5ed80cbe514b035cfc99838a', 'antje416', '6009703d968abd13ebdfbc27', 'Antje', '602bb24175564f7418415c26', None, 0), ('5c004b9ebb81ab0403618dc1', 'claricecloutier', '6007eb661419e5662e28d18d', 'Clarice', '602bb2424c48d5706dda9799', None, 0), ('5eda6809df259163876f4896', 'zaninelli', '5ee86a31b4dd441463ba65d6', 'Luca', '602bb244575c6775f9a91283', '79f8f30cb4eb3e4d1fb4a689316365dd17a0fac5142d4ba7434389ce33cc13fb', 1), ('5edbcc7b78d55673cd5bda17', 'manueltolotti', '5ee8c7329f86d41a917505f5', 'Manuel', '602bb243a088753e0b0ccd26', None, 0), ('5ed8067e73403d50623f87fd', 'marcozaninelli', '5ee869db1e066a18aec6ba98', 'Marco', '6027b3453b34db1f68e548d5', None, 0), ('5ed8f7cfaaaef0624bcb4200', 'valentinatasin', '5ee8c726d3314a3af9a9290f', 'Valentina', '602bb2439500870be27b560e', None, 0), ('5efed0b3bdfd1b6b709ff513', 'vincenzolunelli', '5f02e306e2d1536d1c293e42', 'Vincenz

In [19]:
var = 5
print(var)

5
